In [1]:
import tensorflow as tf
import numpy as np
#import matplotlib.plot as pyplt
import pandas as pd
import time
localtime = time.asctime( time.localtime(time.time()) )
print (localtime)

Thu Apr 14 21:15:18 2016


Read train and test data

In [2]:
train_file = 'train.csv'
test_file = 'test.csv'
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)

In [3]:
print (train.columns)
print (train['label'].head())
print (test.shape)

Index([u'label', u'pixel0', u'pixel1', u'pixel2', u'pixel3', u'pixel4',
       u'pixel5', u'pixel6', u'pixel7', u'pixel8',
       ...
       u'pixel774', u'pixel775', u'pixel776', u'pixel777', u'pixel778',
       u'pixel779', u'pixel780', u'pixel781', u'pixel782', u'pixel783'],
      dtype='object', length=785)
0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64
(28000, 784)


In [4]:
image_size = 28
num_labels = 10
if isinstance(train, pd.DataFrame):
    train = train.as_matrix()
if isinstance(test, pd.DataFrame):
    test_dataset = test.as_matrix().astype(np.float32)
#print (train.shape)
    
def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_num, img_size= train.shape
train_size = int(train_num * 0.7)

train_dataset = train[0:train_size,1:]
valid_dataset = train[train_size::,1:]

train_labels_raw = train[0:train_size, 0]
valid_labels_raw = train[train_size::,0]


train_dataset, train_labels = reformat(train_dataset, train_labels_raw)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels_raw)

print (train_dataset.shape)
print (train_labels.shape)
print (type(train_dataset))
print (test_dataset.shape)
print (train_labels[0:5,:])
print (valid_dataset.shape)

(29399, 784)
(29399, 10)
<type 'numpy.ndarray'>
(28000, 784)
[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
(12601, 784)


In [ ]:
from sklearn.svm import SVC
from sklearn import svm
num = 20000
model2 = SVC()
 
model2.fit(train_dataset[0:num,:], train_labels_raw[0:num,])
    
valid_pred = model2.predict(valid_dataset)
print ('Training Nums:',valid_pred.shape[0], 'Training Accuracy:', float(sum(valid_pred==valid_labels_raw)) / float(valid_pred.shape[0]))

In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [6]:
batch_size = 200

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  #tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  print ('Run the model!')

Run the model!


In [7]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      #print("Validation accuracy: %.1f%%" % accuracy(
      #  valid_prediction.eval(), valid_labels))
  #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  print (type(test_prediction))
  test_pred = test_prediction.eval()
  print (type(test_pred))
  print (test_pred.shape)
  out = np.argmax(test_pred, 1)
  print out.shape
  out = pd.DataFrame(out)
  out.to_csv('out.csv')
  print ('done!')
  print ('done')

Initialized
Minibatch loss at step 0: 3649.566895
Minibatch accuracy: 6.5%
Minibatch loss at step 500: 824.608765
Minibatch accuracy: 87.5%
Minibatch loss at step 1000: 588.991577
Minibatch accuracy: 91.0%
Minibatch loss at step 1500: 924.452087
Minibatch accuracy: 85.0%
Minibatch loss at step 2000: 741.771851
Minibatch accuracy: 91.5%
Minibatch loss at step 2500: 895.657043
Minibatch accuracy: 85.0%
Minibatch loss at step 3000: 562.268738
Minibatch accuracy: 93.5%
Minibatch loss at step 3500: 830.433228
Minibatch accuracy: 89.5%
Minibatch loss at step 4000: 6555.941406
Minibatch accuracy: 79.0%
Minibatch loss at step 4500: 1258.187012
Minibatch accuracy: 83.0%
Minibatch loss at step 5000: 1354.845581
Minibatch accuracy: 83.5%
Minibatch loss at step 5500: 256.456177
Minibatch accuracy: 93.0%
Minibatch loss at step 6000: 456.457642
Minibatch accuracy: 91.0%
<class 'tensorflow.python.framework.ops.Tensor'>
<type 'numpy.ndarray'>
(28000, 10)
(28000,)
done!
done


In [10]:
batch_size = 128
n_hidden = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  tf_hidden_dataset = tf.placeholder(tf.float32, shape = (batch_size, n_hidden))
    
  

  # Variables.
  weights_0 = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_hidden]))
  biases_0 = tf.Variable(tf.zeros([n_hidden]))
    
  weights_1 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases_1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_level = tf.matmul(tf_train_dataset, weights_0) + biases_0
  tf_hidden_dataset = tf.nn.relu(hidden_level)
  logits = tf.matmul(tf_hidden_dataset, weights_1) + biases_1
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
  

  def multilayer_perceptron(x, w_0, b_0, w_1, b_1):
        l1 = tf.nn.relu(tf.matmul(x, w_0) + b_0)
        pred = tf.matmul(l1, w_1) + b_1
        return pred
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights_0, biases_0, weights_1, biases_1))
  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights_0, biases_0, weights_1, biases_1))
  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_0) + biases_0), weights_1) + biases_1)
  #test_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_0) + biases_0), weights_1) + biases_1))
  print (r'Run the following model!')

Run the following model!


In [13]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  print (type(train_prediction))
  print (type(test_prediction))
  test_pred = test_prediction.eval()
  print (type(test_pred))
  print (test_pred.shape)
  out = np.argmax(test_pred, 1)
  print out.shape
  out = pd.DataFrame(out)
  out.to_csv('out-single level CNN.csv', header = ['Label'])
  print ('done!')
  print ('done')

Initialized
Minibatch loss at step 0: 56586.421875
Minibatch accuracy: 7.8%
Validation accuracy: 30.1%
Minibatch loss at step 500: 288.070038
Minibatch accuracy: 95.3%
Validation accuracy: 93.4%
Minibatch loss at step 1000: 73.831123
Minibatch accuracy: 97.7%
Validation accuracy: 93.9%
Minibatch loss at step 1500: 12.505539
Minibatch accuracy: 99.2%
Validation accuracy: 94.3%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 94.4%
Minibatch loss at step 2500: 11.752121
Minibatch accuracy: 97.7%
Validation accuracy: 94.4%
Minibatch loss at step 3000: 4.557785
Minibatch accuracy: 99.2%
Validation accuracy: 94.4%
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<type 'numpy.ndarray'>
(28000, 10)
(28000,)
done!
done


In [14]:
batch_size = 256
n_hidden = [512, 512]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  
  tf_test_dataset = tf.constant(test_dataset)
  
  #tf_hidden_dataset = tf.placeholder(tf.float32, shape = (batch_size, n_hidden))
    
  

  # Variables.
  #weights_0 = tf.Variable(
  #  tf.truncated_normal([image_size * image_size, n_hidden]))
  weights = [tf.truncated_normal([image_size * image_size, n_hidden[0]]),
             tf.truncated_normal([n_hidden[0], n_hidden[1]]),
             tf.truncated_normal([n_hidden[1], num_labels])]
  #biases_0 = tf.Variable(tf.zeros([n_hidden]))
  #biases_0 = tf.Variable(tf.truncated_normal([n_hidden]))
  biases = [tf.Variable(tf.zeros([n_hidden[0]])),
            tf.Variable(tf.zeros([n_hidden[1]])), 
            tf.Variable(tf.zeros([num_labels]))]  
  #weights_1 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  #biases_1 = tf.Variable(tf.zeros([num_labels]))
  #biases_1 = tf.Variable(tf.truncated_normal([num_labels]))
  
  # Training computation.
  def multilayer_perceptron(x, w, b):
        h1 = tf.nn.relu(tf.matmul(x,w[0]) + b[0])
        h2 = tf.nn.relu(tf.matmul(h1, w[1]) + b[1])
        pred = tf.matmul(h2, w[2]) + b[2]
        return pred
            
  #hidden_level_1 = tf.matmul(tf_train_dataset, weights_0) + biases_0
  #tf_hidden_dataset = tf.nn.softmax(hidden_level)
  #logits = tf.matmul(tf_hidden_dataset, weights_1) + biases_1,
  logits = multilayer_perceptron(tf_train_dataset, weights, biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.000000001).minimize(loss)
  
  
  
  
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  #valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights_0, biases_0, weights_1, biases_1))
  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights,biases))
  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_0) + biases_0), weights_1) + biases_1)
  #test_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_0) + biases_0), weights_1) + biases_1))
  print (r'Run the following model!')

Run the following model!


In [15]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      #print("Validation accuracy: %.1f%%" % accuracy(
      #  valid_prediction.eval(), valid_labels))
  #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  print (type(train_prediction))
  print (type(test_prediction))
  test_pred = test_prediction.eval()
  print (type(test_pred))
  print (test_pred.shape)
  out = np.argmax(test_pred, 1)
  print out.shape
  out = pd.DataFrame(out)
  out.to_csv('out-two level CNN.csv', header = ['Label'])
  print ('done!')

Initialized
Minibatch loss at step 0: 606609.625000
Minibatch accuracy: 17.2%
Minibatch loss at step 100: 525017.500000
Minibatch accuracy: 12.5%
Minibatch loss at step 200: 643852.437500
Minibatch accuracy: 9.0%
Minibatch loss at step 300: 860628.750000
Minibatch accuracy: 10.9%
Minibatch loss at step 400: 496180.062500
Minibatch accuracy: 19.5%
Minibatch loss at step 500: 530763.000000
Minibatch accuracy: 9.8%
Minibatch loss at step 600: 739653.000000
Minibatch accuracy: 7.8%
Minibatch loss at step 700: 798790.875000
Minibatch accuracy: 11.3%
Minibatch loss at step 800: 733743.750000
Minibatch accuracy: 2.7%
Minibatch loss at step 900: 703986.625000
Minibatch accuracy: 8.2%
Minibatch loss at step 1000: 659720.250000
Minibatch accuracy: 11.7%
Minibatch loss at step 1100: 597081.375000
Minibatch accuracy: 10.9%
Minibatch loss at step 1200: 512912.906250
Minibatch accuracy: 6.6%
Minibatch loss at step 1300: 925025.812500
Minibatch accuracy: 9.4%
Minibatch loss at step 1400: 510685.43750